## Importing Libraries

In [16]:
import pandas as pd
import numpy as np
import torch
import csv
import json
import arrow
import gpytorch
import random
import matplotlib.pyplot as plt
%matplotlib inline
from torch.nn import Module
from gpytorch.kernels import Kernel
from gpytorch.constraints import Positive
from gpytorch.functions import RBFCovariance
from gpytorch.kernels.kernel import Kernel
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from gpytorch.means.mean import Mean                                      
from torch.nn.utils import clip_grad_norm_
from gpytorch.kernels import Kernel
import seaborn as sns
from scipy import interp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_auc_score, roc_curve
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold


## setting the seed for reproducibility

In [17]:
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)


# Main code

In [21]:

def load_and_preprocess_data(df):
    
    """
    Loads the input file and preprocesses it.

    This function reads data from a CSV file, scales selected columns, 
    generates Morton codes for spatial data, and extracts additional 
    attributes for further processing. It prepares the data for machine 
    learning models by transforming it into tensors.

    Parameters:
    file_path (str)

    Returns:
    tuple: A tuple containing:
        - X (torch.Tensor): The predictors - 'cols_to_scale'
        - y (torch.Tensor): The target variable '7_days_lagged_hotspot' .
        - dates (pd.Series): Series of dates corresponding to each record.
        - additional_data (dict): A dictionary containing additional variables
                                  such as 'dates', 'landkreis_id', 'Hotspot', 
                                  'lon', 'lat', and 'CombinedFeature'.
                                  
    """
    
    df = pd.read_csv('path/to/your/inputfile.csv')
    cols_to_scale = [
        'Faelle_neu', 'Inzidenz_7-Tage', 'AnzahlTodesfall', 
        'value_mobility_change', 'Neighbor_Faelle_neu', 
        'Neighbor_Inzidenz_7-Tage', 'Neighbor_AnzahlTodesfall'
    ]
    scaler = StandardScaler()
    df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    
    dates = pd.to_datetime(df['Meldedatum'], format='%d-%m-%Y')
    morton_codes = calculate_morton_codes(df["longitude"].values, df["latitude"].values)
    df['Week'] = pd.to_datetime(df['Meldedatum'], format='%d-%m-%Y').dt.isocalendar().week
    df['CombinedFeature'] = [f'{morton_code}_{week}' for morton_code, week in zip(morton_codes, df['Week'])]
    
    additional_data = {
        "dates": dates,
        "landkreis_id": df["LandKreis_id"].values,
        "Hotspot": df["Hotspot"].values,
        "lon": df["longitude"].values,
        "lat": df["latitude"].values,
        "CombinedFeature": df['CombinedFeature'].values
    }


    X = torch.tensor(df[cols_to_scale].values, dtype=torch.float32)
    y = torch.tensor(df['7_days_lagged_hotspot'].values, dtype=torch.float32)

    return X, y, dates, additional_data

    
class SpatioTemporalGPModel(gpytorch.models.ApproximateGP):
    
    def __init__(self, inducing_points):
        
         
        """
            1. input -> inducing_points which is the points to speed up computations in the Gaussian Process            
            2. Set up the variational distribution using CholeskyVariationalDistribution and the given inducing points' size.            
            3. Set up the mean_module to use a constant mean using gpytorch.means.ConstantMean().            
            4. spatial and temporal covariance kernel establised using scale kernel built on Radial Basis Function (RBF) kernel.
                
        """       
        
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2)
        )
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.spatial_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.temporal_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        
        """
            1. Extract spatial features from x, denoted as spatial_x. These are columns from index 4 to 6 ('Neighbor_Faelle_neu','Neighbor_Inzidenz_7-Tage','Neighbor_AnzahlTodesfall)            
            2. Extract temporal features from x, denoted as temporal_x. These are columns from index 0 to 3('Faelle_neu', 'Inzidenz_7-Tage', 'AnzahlTodesfall','value_mobility_change')
        
        """
               
        mean_x = self.mean_module(x)
        spatial_x = x[:, 3:7]   
        temporal_x = x[:, :4]    
        spatial_covar_x = self.spatial_covar_module(spatial_x)
        temporal_covar_x = self.temporal_covar_module(temporal_x)
        covar_x = spatial_covar_x * temporal_covar_x 
        covar_x = covar_x + torch.eye(x.size(0)) * 1e-3
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

   
def train_Bernoulli(
    model, bernoulli_likelihood, train_loader, val_loader, n, 
    num_epochs = 16,
    ngd_lr     = 1e-7, 
    adam_lr    = 1e-4,
    print_iter = 100,
    modelname  = "STVAmean-RBFkernel",
    l2_lambda  = 0.1):

    """
    
    Trains STGP model with Bernoulli likelihood using provided data loaders.

    Parameters:
    
    model : gpytorch.models.ApproximateGP  - The Gaussian Process model to be trained.
    
    bernoulli_likelihood : gpytorch.likelihoods.BernoulliLikelihood - Bernoulli likelihood associated with the model.
        
    train_loader : torch.utils.data.DataLoader - DataLoader providing batches of training data.
        
    val_loader : torch.utils.data.DataLoader - DataLoader providing batches of validation data.
        
    n : int  - Number of data points.
        
    num_epochs : int - Number of training epochs.
        
    ngd_lr : float, optional - Learning rate for Natural Gradient Descent optimizer.
        
    adam_lr : float, optional - Learning rate for ADAM optimizer.
        
    print_iter : int, optional - Frequency of printing the training progress during epochs.
              
    modelname : str, optional - Name for saving the trained model's weights.
        
    l2_lambda : float, optional - L2 regularization factor.

    Returns:
    
    model : gpytorch.models.ApproximateGP - Trained Gaussian Process model.
    
    bernoulli_likelihood : gpytorch.likelihoods.BernoulliLikelihood  - Trained Bernoulli likelihood.
    
    train_losses : list of floats  - Training losses for each epoch.
    
    val_losses : list of floats  -  Validation losses for each epoch.

    Note:
    1. This function trains the STGP model with Bernoulli likelihood using Natural Gradient Descent (NGD) 
    and ADAM optimizers. Where NGD efficiently tune the variational parameters of the GP model and Adam optimizer for hyperparameters. 
    2. Training loss and validation loss are computed at each epoch, and 
    the model weights are saved. Finally, the training and validation loss curves are plotted.

    """


    model.train()
    bernoulli_likelihood.train()
    
    variational_ngd_optimizer = gpytorch.optim.NGD(
        model.variational_parameters(),
        num_data=n, lr=ngd_lr)
    
    adam_optimizer = torch.optim.Adam([
        {'params': model.hyperparameters()},
        {'params': bernoulli_likelihood.parameters()},
    ], lr=adam_lr)
    
    train_losses = []
    val_losses = []
    
    bernoulli_mll = gpytorch.mlls.VariationalELBO(bernoulli_likelihood, model, num_data=n)  # Evidence Lower Bound Loss function
    
    for i in range(num_epochs):
        epoch_train_loss = 0.0
        for j, data in enumerate(train_loader):
            x_batch, y_hotspot_batch = data
            
            adam_optimizer.zero_grad()
            output = model(x_batch)
            hotspot_loss = -bernoulli_mll(output, y_hotspot_batch)
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            total_loss = hotspot_loss + l2_lambda * l2_norm  # Add L2 regularization
            epoch_train_loss += hotspot_loss.item()
            hotspot_loss.backward()
            adam_optimizer.step() 

            variational_ngd_optimizer.zero_grad()
            output = model(x_batch)
            hotspot_loss = -bernoulli_mll(output, y_hotspot_batch)
            hotspot_loss.backward()
            variational_ngd_optimizer.step()

            if j % print_iter == 0:
                print("[%s] Epoch : %d,\titer : %d,\tloss : %.5e" % (arrow.now(), i, j, hotspot_loss.item()))

        train_losses.append(epoch_train_loss / len(train_loader))
        

        model.eval()
        bernoulli_likelihood.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_batch, y_hotspot_batch in val_loader:
                output = model(x_batch)
                loss = -bernoulli_mll(output, y_hotspot_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        print(f"[{arrow.now()}] Epoch: {i}, Validation loss: {val_loss:.5e}")
        torch.save(model.state_dict(), "saved_models/%s.pth" % modelname)

        model.train()
        bernoulli_likelihood.train()
    

    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Losses')
    plt.show()
    
    return model, bernoulli_likelihood, train_losses, val_losses
    

def hotspot_prediction(model, bernoulli_likelihood, data_loader):

    """
    Predict hotspots using the given model and bernoulli likelihood.
    
    - This function evaluates the model on the provided data loader and returns
    the mean of the bernoulli likelihood for each input in the data loader.
    
    Parameters:
    - model (gpytorch.models.ApproximateGP): A trained Gaussian process model.
    - bernoulli_likelihood (gpytorch.likelihoods.BernoulliLikelihood): The Bernoulli likelihood associated with the model.
    - data_loader (torch.utils.data.DataLoader): DataLoader containing the input data for prediction.
    
    Returns:
    - means (numpy.array): The mean of the Bernoulli likelihood for each input. These values can be interpreted as the probabilities 
                           of the inputs being hotspots.
    
    Usage:
    The function is used after training to make predictions on unseen data.
    
    """

    model.eval()
    bernoulli_likelihood.eval()
    with torch.no_grad():
       
        means = [ 
            bernoulli_likelihood(model(x_batch)).mean.detach().numpy() 
            for x_batch, _ in data_loader ] 
    means = np.concatenate(means, axis=0)
    
    return means
    

def interleave_bits(x, y):
        
    """
    Interleaves the bits of two integers. i.e. the process of alternating the bits from two integers.

    This function combines the bits of two integers (x and y) in an interleaved manner. 
    It is commonly used in computing Morton codes or Z-order curve values, which are useful 
    for space-filling curves in multidimensional arrays. The function processes 32 bits from 
    each number, interleaving them into a single 64-bit integer.

    Parameters:
    x (int): The first integer representing the x-coordinate in a 2D space.
    y (int): The second integer representing the y-coordinate in a 2D space.

    Returns:
    int: A 64-bit integer with the bits of x and y interleaved.
    
    """
    
    z = 0
    for i in range(32):
        z |= (x & (1 << i)) << i | (y & (1 << i)) << (i + 1)
    return z
    

def calculate_morton_codes(longitudes, latitudes):
    
    """
    
    Calculates Morton codes for a set of coordinates.

    This function scales and converts geographic coordinates (longitudes and latitudes) to 
    integers and then computes their Morton codes. Morton codes are useful for spatial indexing.

    Parameters:
    longitudes & latitudes

    Returns:
    list: A list of Morton codes corresponding to the input coordinates.
    
    """
    
    scaled_lons = ((longitudes - longitudes.min()) / (longitudes.max() - longitudes.min()) * 1e6).astype(int)
    scaled_lats = ((latitudes - latitudes.min()) / (latitudes.max() - latitudes.min()) * 1e6).astype(int)
    return [interleave_bits(lon, lat) for lon, lat in zip(scaled_lons, scaled_lats)]
    

def train_evaluate_loop(X, y, additional_data, n_splits=3):
    
    """
    Trains and evaluates a SpatioTemporal Gaussian Process Model using stratified k-fold cross-validation with 'combined feature (Morton encoding)' as a key.

    parameters:
        X (Tensor): Input features tensor.
        y (Tensor): Target variable tensor.
        additional_data (dict): Dictionary containing additional data like dates, landkreis_id, hotspot, longitude, and latitude.
        n_splits (int): Number of splits for cross-validation.

    Returns:
        dict: Dictionary containing model evaluation metrics like precision, recall, accuracy, F1 score, ROC AUC, and confusion matrices.

    This function performs the following key steps:
    
    - Stratified K-Fold with Morton encoding splitting of the data (training, validation and test).
    - Data resampling using SMOTE (applied only for train set) for handling class imbalance.
    - Training a Gaussian Process Model on the training set.
    - Evaluating the metrics like precision, recall, F1-score, ROC AUC, etc. on the validation and test sets.
    - Collecting the result data (csv file) which contains Actual value (target variable) and Predicted value with its corresponding date, landkreis_ids, latitude and longitude. This result data has been collected to visualize the result. 
    - Returning evaluation metrics.

    This function is an important part of a machine learning workflow in scenarios involving 
    spatiotemporal data, ensuring a robust evaluation of the model.
    
    """
    
    gpytorch.settings.cholesky_jitter(1e-1)

    all_precision_model, all_recall_model, all_accuracy_model, all_f1_model = [], [], [], []
    all_precision_goal, all_recall_goal, all_accuracy_goal, all_f1_goal = [], [], [], []
    all_roc_auc_model, all_roc_auc_goal = [], []

    X_numpy = X.cpu().numpy()
    y_numpy = y.cpu().numpy()

    all_dates = []
    all_y_test = []
    all_predicted_probs = []
    all_train_losses = []
    all_val_losses = []
    result_data = []
    all_test_predictions_binary = []
    all_conf_matrices_goal = []
    cumulative_cm_goal = np.zeros((2, 2))
    skf = StratifiedKFold(n_splits, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(X_numpy, additional_data['CombinedFeature']):
        X_tr_va, X_test = X[train_index], X[test_index]
        y_tr_va, y_test = y[train_index], y[test_index]
        train_index, val_index = train_test_split(np.arange(len(X_tr_va)), test_size=0.20, random_state=42)
        X_train, y_train = X_tr_va[train_index], y_tr_va[train_index]
        X_val, y_val = X_tr_va[val_index], y_tr_va[val_index]        
                          
        X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)

        X_train = torch.tensor(X_train_resampled, dtype=torch.float32)
        y_train = torch.tensor(y_train_resampled, dtype=torch.float32)

        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)  
        val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)    
        test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)
      
        inducing_points = X_train[:500, :]
        model = SpatioTemporalGPModel(inducing_points)
        likelihood = gpytorch.likelihoods.BernoulliLikelihood()
       
        model, likelihood, train_losses, val_losses = train_Bernoulli(model, likelihood, train_loader, val_loader, n=len(X_train))
        all_train_losses.extend(train_losses)
        all_val_losses.extend(val_losses)
        all_dates.extend(additional_data['dates'][test_index].tolist())

        test_predictions = hotspot_prediction(model, likelihood, test_loader)
       
        test_predictions_binary = (test_predictions > 0.5).astype(int)
    

        all_test_predictions_binary.extend(test_predictions_binary.tolist())
        y_test_actual_hotspot = additional_data['Hotspot'][test_index].astype(int)
        cm_goal_fold = confusion_matrix(y_test_actual_hotspot, test_predictions_binary)
        cumulative_cm_goal += cm_goal_fold

        all_y_test.extend(y_test.tolist())  
        all_predicted_probs.extend(test_predictions)

        precision_model = precision_score(y_test.cpu().numpy(), test_predictions_binary)
        recall_model = recall_score(y_test.cpu().numpy(), test_predictions_binary)
        accuracy_model = accuracy_score(y_test.cpu().numpy(), test_predictions_binary)
        f1_model = f1_score(y_test.cpu().numpy(), test_predictions_binary)

        precision_goal = precision_score(y_test_actual_hotspot, test_predictions_binary)
        recall_goal = recall_score(y_test_actual_hotspot, test_predictions_binary)
        accuracy_goal = accuracy_score(y_test_actual_hotspot, test_predictions_binary)
        f1_goal = f1_score(y_test_actual_hotspot, test_predictions_binary)

        all_precision_model.append(precision_model)
        all_recall_model.append(recall_model)
        all_accuracy_model.append(accuracy_model)
        all_f1_model.append(f1_model)

        all_precision_goal.append(precision_goal)
        all_recall_goal.append(recall_goal)
        all_accuracy_goal.append(accuracy_goal)
        all_f1_goal.append(f1_goal)
      
        for idx in range(len(test_predictions)):
            index = test_index[idx]
            result_data.append({
                'Date': additional_data['dates'].iloc[index],
                'Actual Value': y_test.cpu().numpy()[idx],
                'Predicted Value': test_predictions[idx],
                'Predicted Value Binary': test_predictions_binary[idx],
                'Landkreis_id': additional_data['landkreis_id'][index],
                'Hotspot': additional_data['Hotspot'][index],
                'Longitude': additional_data['lon'][index],
                'Latitude': additional_data['lat'][index]
            })

    avg_precision_model = np.mean(all_precision_model)
    avg_recall_model = np.mean(all_recall_model)
    avg_accuracy_model = np.mean(all_accuracy_model)
    avg_f1_model = np.mean(all_f1_model)

    avg_precision_goal = np.mean(all_precision_goal)
    avg_recall_goal = np.mean(all_recall_goal)
    avg_accuracy_goal = np.mean(all_accuracy_goal)
    avg_f1_goal = np.mean(all_f1_goal)
   
    all_test_predictions_binary = np.array(all_test_predictions_binary)

    all_y_test = np.array(all_y_test)
    all_predicted_probs = np.array(all_predicted_probs)
    avg_cm_goal = cumulative_cm_goal / n_splits

    fpr_model, tpr_model, thresholds_model = roc_curve(all_y_test, all_predicted_probs)
    roc_auc_model = auc(fpr_model, tpr_model)
    all_roc_auc_model.append(roc_auc_model)

    avg_roc_auc_model = np.mean(all_roc_auc_model)

    plt.figure()
    plt.plot(fpr_model, tpr_model, color='darkorange', lw=2, label=f'Model ROC curve (area = {avg_roc_auc_model:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve - Model')
    plt.legend(loc="lower right")
    plt.show()

    print(f"Model Average Precision: {avg_precision_model:.4f}")
    print(f"Model Average Recall: {avg_recall_model:.4f}")
    print(f"Model Average Accuracy: {avg_accuracy_model:.4f}")
    print(f"Model Average F1 Score: {avg_f1_model:.4f}")
    print(f"Model Average ROC AUC: {avg_roc_auc_model:.4f}")

    print(f"Goal Average Precision: {avg_precision_goal:.4f}")
    print(f"Goal Average Recall: {avg_recall_goal:.4f}")
    print(f"Goal Average Accuracy: {avg_accuracy_goal:.4f}")
    print(f"Goal Average F1 Score: {avg_f1_goal:.4f}")

    cm_model = confusion_matrix(all_y_test, all_test_predictions_binary)
    print("Model Confusion Matrix:")
    print(cm_model)
 
    result_df = pd.DataFrame(result_data)
    result_df.to_csv('path/to/store/your/output/datafile.csv', index=False)
    
    return {
        "model_precision": avg_precision_model,
        "model_recall": avg_recall_model,
        "model_accuracy": avg_accuracy_model,
        "model_f1": avg_f1_model,
        "model_roc_auc": avg_roc_auc_model,
        "model_confusion_matrix": cm_model,
        "goal_precision": avg_precision_goal,
        "goal_recall": avg_recall_goal,
        "goal_accuracy": avg_accuracy_goal,
        "goal_f1": avg_f1_goal,
        "goal_confusion_matrix": cumulative_cm_goal
        
    }


if __name__ == "__main__":
    
    
    """
    Main execution block to load data, train and evaluate the model, and visualize the results.

    This block:
    - Loads the data from the specified CSV file path.
    - Preprocesses the data and splits it into features (X), labels (y), and additional data needed for the model.
    - Trains the model using the SpatioTemporalGPModel and evaluates it using various metrics like precision, recall, accuracy, and F1-score.
    - Visualizes the model's confusion matrix (Target variable - '7-days lagged hotspot' and its prediction) and the goal which is actual Hotspot column vs. predicted 7-days lagged hotspot -  comparison for better understanding and analysis.

    The path to the input data file should be specified in place of 'path/to/your/inputfile.csv'
    
    """

    X, y, dates, additional_data = load_and_preprocess_data('path/to/your/inputfile.csv')   
    results = train_evaluate_loop(X, y, additional_data)
    avg_precision = results["model_precision"]
    avg_recall = results["model_recall"]
    avg_accuracy = results["model_accuracy"]
    avg_f1 = results["model_f1"]

    plt.figure(figsize=(5, 4))
    sns.heatmap(results["model_confusion_matrix"], annot=True, fmt='g', cmap='Blues', 
                xticklabels=['Predicted 0', 'Predicted 1'], 
                yticklabels=['Actual 0', 'Actual 1'])
    plt.title("Model Confusion Matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    plt.figure(figsize=(5, 4))
    sns.heatmap(results["goal_confusion_matrix"], annot=True, fmt='g', cmap='Blues', 
                xticklabels=['Non-Hotspot', 'Hotspot'],
                yticklabels=['Non-Hotspot', 'Hotspot'])
    plt.title("actual vs predicted hotspot for next week")
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

